In [4]:
from threading import Thread
from multiprocessing import Process
import multiprocessing
import numpy as np
import subprocess
import signal
import time
import os
import sys

In [ ]:
def start_stress(num_cores, time):
    print("Stressing ",num_cores," cores for ",time," seconds")
    t= str(time)+'s'
    cmd= 'stress '+'-c '+str(num_cores)+' -t '+ t
    pid_stress = subprocess.Popen(['stress', '-c', str(num_cores), "-t", t],
                        stdout=subprocess.PIPE, stderr=subprocess.PIPE, preexec_fn=os.setsid)
    return pid_stress

In [ ]:
def add_data():
    global model_num
    global name_dataSet
    global str_header
    global str_Wt
    global str_Ct
    global str_Ac
    global str_Pr
    global str_Re
    global str_Line
    
    name_file= ""
    
    if(model_num == 0):
        str_header[0]= "Logistic Regression"
        name_file= "_pcLR"
    elif(model_num == 1):
        str_header[0]= "Support Vector Machine"
        name_file= "_pcSVM"
    elif(model_num == 2):
        str_header[0]= "Gradient Boosting"
        name_file= "_pcGB"
    elif(model_num == 3):
        str_header[0]= "Random Forest"
        name_file= "_pcRF"
        
    data= np.array([str_header, str_Wt, str_Ct, str_Ac, str_Pr, str_Re])
    
    name_file=  name_dataSet+name_file+".csv"  
    
    np.savetxt(name_file,data,delimiter=",",fmt='%s')

In [ ]:
def get_WallT(line, str_Wt):
    
    wall_time= line.replace("real\t","").split("m")
    minutes= str(wall_time[0]+"m")
    sec= float(wall_time[1].replace("s",""))
    sec= str(round(sec, 2))+"s "
    
    str_wallTime= " "+minutes+sec
        
    str_Wt.append(str_wallTime)

In [ ]:
def get_CpuT(user_line, sys_line, str_Ct):
    
    user_time= user_line.split("\t")[1]
    user_time= user_time.split("m")
    
    sys_time= sys_line.split("\t")[1]
    sys_time= sys_time.split("m")
    
    minutes= int(user_time[0])+int(sys_time[0])
    minutes= str(minutes)+"m"
    sec= float(user_time[1].replace("s",""))+float(sys_time[1].replace("s",""))
    sec= str(round(sec,2))+"s "
    
    str_cpuTime= minutes+sec
    
    str_Ct.append(str_cpuTime)

In [ ]:
def get_APR(output, str_Ac, str_Pr, str_Re):
    output=output.split("Accuracy")[1]#para quitar caracteres extranos
    output=output.split("\n")
    
    ac_output= output[0].split(":")[1]
    pre_output= output[1].split(":")[1]
    rec_output= output[2].split(":")[1]
    
    str_Ac.append(ac_output)
    str_Pr.append(pre_output)
    str_Re.append(rec_output)

In [ ]:
def get_data(m_name, output, times):
    global num_cores
    global list_cpus
    global name_dataset
    global str_header
    global str_Wt
    global str_Ct
    global str_Ac
    global str_Pr
    global str_Re
    
    output= output.decode("utf-8")
    times= times.decode("utf-8")
    
    if(num_cores== list_cpus[0]):
        str_header= ["","Idle"]
        str_Wt= ["Wall time"]
        str_Ct= ["Cpu time"]
        str_Ac= ["Accuracy"]
        str_Pr= ["Precision"]
        str_Re= ["Recall"]
        
    elif(num_cores > 0):
        str_header.append((str(num_cores)+" stress"))
    
    lines= times.split("\n")
    
    get_WallT(lines[1], str_Wt)
    
    get_CpuT(lines[2], lines[3], str_Ct)
    
    get_APR(output, str_Ac, str_Pr, str_Re)
    
    
    if(num_cores==list_cpus[-1]):
        add_data()

In [ ]:
def model(model_num):
    global dataSet
    global num_cores
    global name_dataset
    
        
    m_name= ""
    
    if model_num == 0:
        m_name= "Logistic Regression"
        print("Executing ",m_name)
        output, times = subprocess.Popen(['bash', '-c', 'time "$@" 2>&1', '_',"ipython", './Modelos/lRegression.ipynb', dataSet],
                        stdout=subprocess.PIPE, stderr=subprocess.PIPE).communicate()
        
    if model_num == 1:
        m_name= "Maquinas de soporte vectorial"
        print("Executing ",m_name)
        m_name= "\n"+m_name
        output, times = subprocess.Popen(['bash', '-c', 'time "$@" 2>&1', '_',"ipython", './Modelos/svm.ipynb', dataSet],
                        stdout=subprocess.PIPE, stderr=subprocess.PIPE).communicate()

    if model_num == 2:
        m_name= "Gradient Boosting"
        print("Executing ",m_name)
        output, times = subprocess.Popen(['bash', '-c', 'time "$@" 2>&1', '_',"ipython", './Modelos/gBoosting.ipynb', dataSet],
                        stdout=subprocess.PIPE, stderr=subprocess.PIPE).communicate()
    
    if model_num == 3:
        m_name= "Random Forest"
        print("Executing ",m_name)
        output, times = subprocess.Popen(['bash', '-c', 'time "$@" 2>&1', '_',"ipython", './Modelos/rForest_pc.ipynb', dataSet],
                        stdout=subprocess.PIPE, stderr=subprocess.PIPE).communicate()
        
        
    get_data(m_name, output,times)

In [5]:
program= sys.argv[0].split("/")

if(len(program) > 1):
    program= program[-1]

if (len(sys.argv) != 2):
    error=""
    error=error+"Error: wrong number of arguments, execute as: \nipython "+program+" dataset_name"
    sys.exit(error)

SystemExit: Error: wrong number of arguments, execute as: 
python /home/nuriadj/anaconda3/envs/tfg/lib/python3.9/site-packages/ipykernel_launcher.py dataset_name

/home/nuriadj/anaconda3/envs/tfg/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3465: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
#Hacer una version que ejecute de seguido los modelos solo (sin stress)
t= 200
list_cpus= [0,2,4,8]
dataSet= sys.argv[1]
name_dataSet= dataSet.split("/")[-1].split(".csv")[0]

str_header= []
str_Wt= []
str_Ct= []
str_Ac= []
str_Pr= []
str_Re= []

for i in range(4):
    for j in list_cpus:
        num_cores= j
        if(j > 0):
            pid_stress= start_stress(j, t)
        else:
            print("\nCpu Idle")
            
        model_num= i    
        model(model_num)
        
        if j > 0:
            os.killpg(os.getpgid(pid_stress.pid), signal.SIGKILL)
            #pid_stress.communicate()
            #pid_stress.wait()
        
        print("Sleeping for 5 sec...\n")
        time.sleep(5)